In [1]:
import pandas as pd
import matplotlib as mpl
import os
import requests
from tqdm import tqdm
import numpy as np
import random
tqdm.pandas()

<center>Contents:</center><a class="anchor" id="contents"></a>

1. [Synthesize data](#synthesize)

*1.1. Create random DataFrame*

*1.2. insert 10% NaN values to DataFrame*

*1.3. Function to add column with random text*

*1.4. Create column for cohorts*


2. [Read & Import, merge, join, concatenate](#import)

*2.1. Glue few tables to one*

3. [Analysis](#analysis)

4. [Computing acceleration and preprocessing](#preprocessing)

5. [Other](#other)

## Synthesize data
<a class="anchor" id="synthesize"></a>

[оглавление](#contents) | [дальше>](#import)

### 1.1. Create random DataFrame:

In [2]:
random_created_df = pd.DataFrame(np.random.randint(0, 100, size=(1000, 4)),
                                 columns=['a', 'b', 'c', 'd'])

# random_created_df.info()
random_created_df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
a,1000.0,50.108,29.049134,0.0,25.0,49.0,77.0,99.0
b,1000.0,50.455,28.703865,0.0,26.0,51.0,75.0,99.0
c,1000.0,50.436,29.381715,0.0,25.0,52.5,77.0,99.0
d,1000.0,49.482,28.191673,0.0,25.0,50.0,74.0,99.0


### 1.2. Insert 10% NaN values

In [3]:
random_created_df = random_created_df.mask(np.random.random(random_created_df.shape) < .1)

# random_created_df.info()
random_created_df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
a,915.0,49.397814,29.060926,0.0,24.00,48.0,76.0,99.0
b,896.0,50.549107,28.685773,0.0,26.00,51.5,75.0,99.0
c,896.0,50.768973,29.203872,0.0,25.75,53.0,77.0,99.0
d,910.0,49.774725,28.384109,0.0,25.00,50.0,74.0,99.0


### 1.3. Function to add column with random text

In [4]:
%%time

def function_to_apply(i):
    first = requests.get(f'https://random-word-api.herokuapp.com/word').json() # Random Word API
    
#     add word
#     second = requests.get(f'https://random-word-api.herokuapp.com/word').json()
#     third = requests.get(f'https://random-word-api.herokuapp.com/word').json()

    return first[0]
#     return " ".join([first[0], second[0], third[0]])

random_created_df['words'] = random_created_df['d'].apply(function_to_apply)

Wall time: 5min 38s


In [5]:
random_created_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
a,915.0,NaN,NaN,NaN,49.397814,29.060926,0.0,24.0,48.0,76.0,99.0
b,896.0,NaN,NaN,NaN,50.549107,28.685773,0.0,26.0,51.5,75.0,99.0
c,896.0,NaN,NaN,NaN,50.768973,29.203872,0.0,25.75,53.0,77.0,99.0
d,910.0,NaN,NaN,NaN,49.774725,28.384109,0.0,25.0,50.0,74.0,99.0
words,1000,998,roseslug,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.4. Create column for cohorts

In [6]:
col         = 'a'
conditions  = [ random_created_df[col] >= 65, (random_created_df[col] < 65) & (random_created_df[col] >= 40), random_created_df[col] < 40 ]
choices     = [ '65>', '40-65', '<40' ]
    
random_created_df['cohort'] = np.select(conditions, choices, default=np.nan)

In [7]:
random_created_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
a,915.0,NaN,NaN,NaN,49.397814,29.060926,0.0,24.0,48.0,76.0,99.0
b,896.0,NaN,NaN,NaN,50.549107,28.685773,0.0,26.0,51.5,75.0,99.0
c,896.0,NaN,NaN,NaN,50.768973,29.203872,0.0,25.75,53.0,77.0,99.0
d,910.0,NaN,NaN,NaN,49.774725,28.384109,0.0,25.0,50.0,74.0,99.0
words,1000,998,roseslug,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cohort,1000,4,<40,377,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
type(np.nan)

float

In [17]:
random_created_df[random_created_df['cohort'] == 'nan']

,a,b,c,d,words,cohort
21,NaN,10.0,58.0,83.0,varicosed,nan
29,NaN,30.0,85.0,58.0,serotonergic,nan
35,NaN,60.0,43.0,NaN,beautician,nan
45,NaN,31.0,42.0,37.0,centerfolds,nan
54,NaN,24.0,15.0,36.0,handstamp,nan
...,...,...,...,...,...,...
970,NaN,38.0,24.0,80.0,deodorize,nan
975,NaN,81.0,17.0,17.0,lammergeyers,nan
982,NaN,NaN,NaN,47.0,restock,nan
984,NaN,22.0,23.0,41.0,aimlessnesses,nan


## Read & Import, merge, join, concatenate
<a class="anchor" id="import"></a>

[оглавление](#contents) | [дальше>](#join)

Some topics:

[Работа с большими наборами данных](https://habr.com/ru/company/ruvds/blog/442516/)

[Управление памятью](https://habr.com/ru/company/mailru/blog/336156/)

[How upload to notebook files from github (medium.com)](https://medium.com/towards-entrepreneurship/importing-a-csv-file-from-github-in-a-jupyter-notebook-e2c28e7e74a5)

## 2.1. Glue few tables to one

In [ ]:
# показать путь к текщей директории
os.getcwd()

# указываем путь к папке с файлами, которые хотим объединить

folder = r'C:\\Users\\Asus\\GKU\\2505\\all_audio' # insert abs path

# создаем список с названиями файлов
files_names = os.listdir(folder)

#формируем список путей к файлам
files = [os.path.join(folder,f) for f in files_names] 

all_file_frames = [] #сюда будем добавлять прочитанную таблицу 

# цикл сборки таблиц в список
for f in files:
    print('Reading %s'%f)
    tab = pd.read_excel(f)
    all_file_frames.append(tab)

# склеиваем все таблицы в списке
all_frame = pd.concat(all_file_frames,axis=0) #  axis=0 если нужно добавить таблицу снизу и axis=1 если нужно слева

In [ ]:
# Change Pandas Column Names to Lower Case

df= df.rename(columns=str.lower)

## Analysis
<a class="anchor" id="analysis"></a>

[<назад](#join) | [оглавление](#contents) | [дальше>](#preprocessing)

Some topics:

In [ ]:
df.dtypes.value_counts()

см. также Чтение и быстрое преобразование данных:

# create custom describe

In [24]:
w_df = random_created_df.copy()

show_percent_dict = {}

for _ in w_df.columns.tolist():
    percent = (w_df[w_df[_].isna()].shape[0] / w_df.shape[0]) * 100
    show_percent_dict[_] = percent

show_percent_dict = dict(sorted(show_percent_dict.items(), key=lambda x: x[1], reverse=True))
    
print(f'Всего строк в наборе данных: {w_df.shape[0]}\n')

for k,v in show_percent_dict.items():  
    v = str(int(v)) + '%'
    print(f'{v} - доля пропусков в столбце {k}')

Всего строк в наборе данных: 100000

10% - доля пропусков в столбце c
9% - доля пропусков в столбце d
9% - доля пропусков в столбце a
9% - доля пропусков в столбце b


## Computing acceleration and preprocessing
<a class="anchor" id="preprocessing"></a>

[<назад](#analysis) | [оглавление](#contents)

Some topics:

In [48]:
random_created_df.shape

(1000, 4)

In [49]:
# find rows with equal values
random_created_df['equal 4th'] =  random_created_df.iloc[:,0:5].nunique(axis = 1).eq(1)
random_created_df['equal 4th'].value_counts()

False    998
True       2
Name: equal 4th, dtype: int64

In [50]:
# replace some values to  1

for n in list(range(1,1001)):
    if n % 5 == 0:
        random_created_df.loc[n] = 1

In [51]:
random_created_df['equal 4th'] =  random_created_df.iloc[:,0:5].nunique(axis = 1).eq(1)
random_created_df['equal 4th'].value_counts()

False    800
True     201
Name: equal 4th, dtype: int64

### [about `replace` and `map` vs replace](https://stackoverflow.com/questions/20250771/remap-values-in-pandas-column-with-a-dict-preserve-nans)

If your dictionary has more than a couple of keys, using map can be much faster than replace. There are two versions of this approach, depending on whether your dictionary exhaustively maps all possible values (and also whether you want non-matches to keep their values or be converted to NaNs):


example with binary features^

In [ ]:
# Всем добрый день!
# Помогите решить задачку. Имею следующий формат учета продаж.
# Необходимо посчитать тотал по каждому клиенту. 
# В Power Query задачку решил, в питоне не получается.


# df = pd.read_excel('/.../Новая таблица.xlsx')
# pd.DataFrame.from_records(
#     df.apply(lambda x: {value:x[idx+1] for idx, (col_name, value) in enumerate(zip(df.columns, x)) if 'customer' in col_name.lower()}, axis=1).values
#     ).sum(axis=0)

In [ ]:
report.eq(report.iloc[:,5:10], axis=0).all(1)

## Чтение и быстрое преобразование данных:

#### 1. `value_counts` и `select_dtypes` сэкономит кучу времени:

* `df.dtypes.value_counts()`


* `df.select_dtypes(include = ['float64', 'int64'])`

Полезные аргументы `value_counts`:

* `normalize = True` – проверить частоту вместо подсчёта.


* `dropna = False` – включить пропущенные значения в статистику.


* `df['c'].value_counts().reset_index()` – преобразовать таблицу статистики в объект Pandas DataFrame.


* `df['c'].value_counts().reset_index().sort_values(by='index')` – показывать статистику, отсортированную по уникальным значениям в столбце 'c' вместо количества.

#### 2. `map` - команда для простого преобразования данных.
Определяете словарь, в котором «ключами» являются старые значения, а «значениями» – новые значения:

`level_map = {1: 'high', 2: 'medium', 3: 'low'}`

`df['c_level'] = df['c'].map(level_map)`

#### <font color='red'>Как это использовать?</font>

`all_doc.applymap(type)['BIRTH_DATE'].value_counts()
bad = all_doc[all_doc['BIRTH_DATE'].map(lambda x: type(x) == str)]
good = all_doc[all_doc['BIRTH_DATE'].map(lambda x: type(x) != str)]
all_doc_2 = bad.append(good, ignore_index=True)`

#### 3.1.  Выборка и фильтрация:

конечно можно решать задачи через `loc` `iloc`

`data.loc[:,"User"]`

`data.iloc[:5,0]`

В SQL используем SELECT * FROM… WHERE ID в («A001», «C022»,…) и получаем записи с конкретными идентификаторами. Если хотите сделать то же с помощью Python библиотеки Pandas, используйте:


`df_filter = df['ID'].isin(['A001','C022',...])`

`df[df_filter]`


#### 3.2. [numpy.where](https://numpy.org/doc/stable/reference/generated/numpy.where.html) для работы со значениями:

`numpy.where(data['Revenue']>400,1,0)`

`numpy.where(data['Revenue']<data['Expense'],1,2)`

#### [3.3. Еще 3 полезных метода](https://m.habr.com/ru/company/ruvds/blog/479276/)

#### 3.4. `datetime` processing

In [ ]:
# convert to date from string

# нормализуем и форматируем все даты
datecols = [col for col in pp.columns if re.search('(?i)Дат(а|у)', col)]
gooddatecols = []
baddatecols = []
for datecol in datecols:
  if pp[datecol].dtypes == 'datetime64[ns]':
    gooddatecols.append(datecol)
  else:
    baddatecols.append(datecol)
    
def normalize_date(series):
  def trydate(arg):
    try:
      if pd.isna(arg): return pd.NaT
      if type(arg) == datetime.datetime:
        return pd.to_datetime(arg).normalize()
      elif type(arg) == str:
        match = re.search(r'\d{2}(.)\d{2}\1\d{4}', arg)
        if match:
          delimiter = match.group(1)
          try: return pd.to_datetime(match.group(), format=f'%d{delimiter}%m{delimiter}%Y')
          except: return False
        else: return arg
    except: 
      return False
  if series.name in gooddatecols:
    return series.dt.normalize()
  else:
    return series.apply(trydate)

for datecol in datecols:
  pp[datecol] = normalize_date(pp[datecol])








%%time
for _ in all_frame.columns.tolist():
    if 'Дата' in _:
        all_frame[_] = pd.to_datetime(all_frame[_], errors='ignore') 

all_frame.to_excel(f'направления_к_онколгу_из_облака_BD_1194_на_ОТ_РАЗРАБОВ_{today}.xlsx', index=False)

In [ ]:
# date formatting
# everytime check?????????????????????????????????????????????

# = '2020-01-15'
first_date_type_list = ['Биопсия - Дата проведения биопсии',
                        'Специализированное лечение - Плановая дата проведения лекарственного лечения']

# = '1997-04-05 00:00:00'
dob = ['общая информация - ДР пациента']

# = '1997-04-05T00:00:00+04:00'
second_date_type_list = ['общая информация - Дата и время подписания протокола',
                         'Считает себя больным с ',
                        'Диагноз - Дата устанвоки диагноза',
                        'Новый диагноз - Дата установления нового диагноза',
                        'Онкоконсилиум - Плановая дата проведения онкоконсилиума',                         
                        'Вложенный диагноз на первичном приеме (1) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (2) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (3) - Дата установления диагноза',
                        'Вложенный диагноз на первичном приеме (4) - Дата установления диагноза',
                        'Множественная опухоль(1) - Дата установления диагноза', 
                        # понять почему остался постфикс в конце
                        'Множественная опухоль(2) - Дата установления диагноза',
                        'Множественная опухоль(3) - Дата установления диагноза',
                        'Множественная опухоль(4) - Дата установления диагноза']

date_cols = first_date_type_list+dob+second_date_type_list

# check ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
total_data[date_cols].sample(10)
# total_data[date_cols].info()

%%time
total_data_copy = total_data.copy()

for _ in total_data_copy.columns.tolist():
    if _ in date_cols:
        total_data_copy[_] = total_data_copy[_].fillna(pd.NaT)
        if _ == 'общая информация - ДР пациента':
            total_data_copy[_] = pd.to_datetime(total_data_copy[_], format = '%Y-%m-%d %H:%M:%S', errors='ignore').dt.date
        elif _ in second_date_type_list:
            total_data_copy[_] = total_data_copy[_].str.replace(r'T.+','')
            total_data_copy[_] = pd.to_datetime(total_data_copy[_], format = '%Y-%m-%d', errors='ignore').dt.date
        elif _ in total_data:
            total_data_copy[_] = pd.to_datetime(total_data_copy[_], format = '%Y-%m-%d', errors='ignore').dt.date
            
total_data_copy[date_cols].sample(10)

#### 3.5. `groupby` для объединения значений в строках столбца

full_explorations_df = full_explorations_df.merge(full_explorations_df.groupby(['локализация'])['ЛИО'].apply(lambda x: ', '.join(x)).reset_index(),
                                                  how='left',
                                                  left_on = 'локализация',
                                                  right_on = 'локализация')
full_explorations_df.sample(2).T

## Решение задач из мира Excel

[источник](https://habr.com/ru/company/ruvds/blog/500426/)

#### 0. Рукотворные ID:

`
all_audio['дата рождения2'] = all_audio['дата рождения'].dt.strftime('%d.%m.%Y')
all_audio['ФИО_строкой'] = all_audio['фамилия'] + all_audio['имя'] + all_audio['отчество'] + all_audio['дата рождения2']
all_audio['ФИО_строкой'] = all_audio['ФИО_строкой'].str.replace('.','',regex=False)
all_audio['ФИО_строкой'] = all_audio['ФИО_строкой'].str.replace(' ','',regex=False)
all_audio['ФИО_строкой'] = all_audio['ФИО_строкой'].str.replace('-','',regex=False)
all_audio['ФИО_строкой'] = all_audio['ФИО_строкой'].str.upper()
`

#### и удаление дубликатов по сценарию:

`rez_true = rez2.sort_values('дата и время приема', ascending=False)
rez_true = rez_true.drop_duplicates(subset='idn', keep='first')`

#### 1. Тернарный оператор (конструкция `if`) для создания нового признака/столбца на основе значений уже имеющиегося. Реализация с помощью `list comprehension`:

`sales['MoreThan500'] = ['Yes' if x > 500 else 'No' for x in sales['Sales']]`

#### 2. `VLOOKUP (ВПР)`, решается с помощью функции [`merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html):

`sales = pd.merge(sales, states, how='left', on='City')`

1. Первый аргумент метода merge — это исходный датафрейм.


2. Второй аргумент — это датафрейм, в котором мы ищем значения.


3. Аргумент `how` (`left, right, outer, inner`, default=`inner`) указывает на то, как именно мы хотим соединить данные.


4. Аргумент `on` указывает на переменную, по которой нужно выполнить соединение (тут ещё можно использовать аргументы `left_on` и `right_on`, нужные в том случае, если интересующие нас данные в разных датафреймах названы по-разному).

#### 3. Pivot table:


`sales.pivot_table(index = 'City', values = 'Sales', aggfunc = 'sum')`

1. Здесь мы используем метод `sales.pivot_table`, сообщая pandas о том, что мы хотим создать сводную таблицу, основанную на датафрейме sales.


2. Аргумент `index` указывает на столбец, по которому мы хотим агрегировать данные.


3. Аргумент `values` указывает на то, какие значения мы собираемся агрегировать.


4. Аргумент `aggfunc` задаёт функцию, которую мы хотим использовать при обработке значений (`mean, max, min...`).

#### 4. [Styling](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

#### 5. Заменить PowerQwery:

**было:** входящий формат данных (один айди - несколькострок, меняются 2 последних признака)
<img src="photo_2020-05-27_12-16-44.jpg" style="width: 1000px"/>


**стало:** сделать сводную по дате на признак динамики
<img src="photo_2020-05-27_12-16-52.jpg" style="width: 1000px"/>
    
    
    
`
table = pd.pivot_table(a,
                       values=['статус пациента', 'Динамика заболевания'],
                       index=['сцепка', 'id_emias','полис омс'],
                       lumns=['Дата звонка'], aggfunc=lambda x: ' '.join(x),
                       aggfunc='first') #если на пересечении только одно текстовое значение
`

#### 6. [Смещение данных `shift`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html)


Есть DataFrame. Есть колонка Date, и 2 колонки со значениями Col1 и Col2.
Задача: нужно запилить колонку Col3, которая в случае, если значение col1 == 1 и col2 == 1  (тут я все сделал, это просто), либо если значение из df['col2'].iloc[x]  == 1 и df['col1'].iloc[x+1] == 1, то проставляется 1 в Col3.


**Пример: в таблице ниже за 01/05/20 в Col2 стоит 1,  а за 01/06/20 в Col1 стоит 1, то нужно в новой колонке Col3 поставить 1 на дате 01/05/20.**


<img src="3th.jpg" style="width: 400px"/>


`if df['col1']== 1 and  df['col2'].shift(1) ==1: 1`

#### 7. `crosstab`

In [ ]:
pd.crosstab(a,b)

#### 8. drop-down list

In [ ]:
%%time

wb = Workbook()
ws_zero = wb.active

for r in dataframe_to_rows(bullets, index=False, header=False):
    ws_zero.append(r)

test_list = list(ws_zero.values)
true_valid_list = []
for _ in test_list:
    _ = list(filter(None.__ne__, _))
    true_valid_list.append(_)
    
ws = wb.create_sheet('New Sheet')

# make header
for number in range(1,len(true_input_col_name)+1):
    ws.cell(row=1,column=number,value=true_input_col_name[number-1])

# filling position in MTZ - floor
for num, key in enumerate(mtz_df['этаж'].tolist()):
    ws.cell(row=num+2,column=1,value=key)
    
# filling position in MTZ - position
for num, key in enumerate(mtz_df['ПОЗИЦИЯ'].tolist()):
    ws.cell(row=num+2,column=2,value=key)   

# filling position in MTZ - position number
for num, key in enumerate(mtz_df['ПО_ПЕРЕЧНЮ'].tolist()):
    ws.cell(row=num+2,column=3,value=key)  
    
# filling position in MTZ - items
for num, key in enumerate(mtz_df['НАИМЕНОВАНИЕ'].tolist()):
    ws.cell(row=num+2,column=4,value=key)

# #############################################################
# ws_new = wb.create_sheet('New Sheet_new')

# data_val = DataValidation(type="list",formula1='=Sheet!$1:$1') #You can change =$A:$A with a smaller range like =A1:A9
# ws_new.add_data_validation(data_val)

# data_val.add(ws_new["B1"]) #If you go to the cell B1 you will find a drop down list with all the values from the column A    

str_from_key = ''
# filling DataValidation in MTZ
for num, key in enumerate(true_valid_list):
    str_from_key = ','.join(key)
    data_val = DataValidation(type="list", formula1=str_from_key, allow_blank=True)
    ws_zero.add_data_validation(data_val)
    data_val.add(ws[f'E{num+2}'])
    
    
# from openpyxl.utils import quote_sheetname
# dv = DataValidation(type="list",
#                     formula1="Sheet!$1:$1".format(quote_sheetname('Sheet')))
    
    
# for _ in list(compare_dict.keys()):
#     for num, key in enumerate(compare_dict.keys()):
#         data_val = DataValidation(type="list",formula1=compare_dict[key])
#         ws.cell(row=num+2,column=4,value=key)
        
# ws.cell(column=1, row=new_line_num, value=row[0].value)

# data_val = DataValidation(type="list",formula1='=$A:$A') #You can change =$A:$A with a smaller range like =A1:A9
# ws.add_data_validation(data_val)
# data_val.add(ws["B1"])
        
wb.save('Test_01_v_18.09.2021.xlsx')

## Ускорения расчетов `pandas`:

In [15]:
del df, new_df

In [40]:
# считаем среднюю длину слова
def mean_word_len(line):
    count = [len(i) for i in line.split()]
    res = sum(count) / len(count)
    return res

def compute_avg_word(df):
    return df['words'].apply(mean_word_len)

In [48]:
%timeit compute_avg_word(new_df)

1.24 ms ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### 1. Multiprocessing (default pack)

In [46]:
import multiprocessing
from multiprocessing import Pool

# поределяем кол-во ядер процессора
multiprocessing.cpu_count()

4

In [47]:
pool = Pool(multiprocessing.cpu_count())

In [50]:
def apply_parallel(df, func):
    # делим датафрейм на части
    df_split = np.array_split(df, n_cores)
    # считаем метрики для каждого и соединяем обратно
    df = pd.concat(pool.map(func, df_split))
    return df

In [ ]:
%timeit apply_parallel(new_df, compute_avg_word)

* Ускорение в 2-3 раза;


* Использовать распараллеливание на маленьких данных — плохая идея, т.к накладные расходы на межпроцессорное взаимодействие превышают выигрыш по времени.

#### Создадим 2-ую таблицу в 100 000 строк и 4 колонки, заполненную случайными числами от 0 до 100:

In [5]:
104818 / 239930

0.4368690868169883

In [4]:
df2 = pd.DataFrame(np.random.randint(0,100,size=(100000, 4)),columns=['a', 'b', 'c', 'd'])
df2.shape[0]

100000

In [ ]:
# функция для создания новой колонки
def multiply(x):
    return x * 5

In [5]:
# наша функция
%timeit df2['new_col'] = df2['a'].apply(multiply)

62.6 ms ± 8.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
# встроенная имплементация Pandas
%timeit df2['new_col'] = df2['a'] * 5

1.24 ms ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
# возводим значения строки в квадрат и берем их среднее 
def square_mean(row):
    row = np.power(row, 2)
    return np.mean(row)

In [10]:
# применение:
%timeit df2['new_col'] = df.apply(square_mean, axis=1)

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until


17.8 s ± 3.96 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### 2.  [`numba`](https://habr.com/ru/post/503726/)

* Возможно добиться ускорения в тысячи раз;


* Можно использовать далеко не везде, в основном для оптимизации математических операций;


* Поддерживает не все возможности `python` и `numpy`.

In [ ]:
import numba

In [7]:
# наша функция с оптимизированная `numba`
@numba.vectorize
def multiply_numba(x):
    return x * 5

# мы отдаем весь вектор значений, чтобы numba сам провел оптимизацию цикла
%timeit df2['new_col'] = multiply_numba(df2['a'].to_numpy())

767 µs ± 350 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# numba не умеет работать с примитивами pandas (Dataframe, Series и тд.)
# поэтому мы даем ей двумерный массив numpy
@numba.njit
def square_mean_numba(arr):
    res = np.empty(arr.shape[0])
    arr = np.power(arr, 2)
    for i in range(arr.shape[0]):
        res[i] = np.mean(arr[i])
    return res

In [13]:
# применение:
%timeit df['new_col'] = square_mean_numba(df.to_numpy())

9.4 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### 3. [`pandarallel`](https://habr.com/ru/post/498904/)

In [5]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[Выполним данную задачу](#create_df)[<font color='red'> функциями `pandarallel`:</font>](https://github.com/nalepae/pandarallel/blob/master/docs/examples.ipynb)

In [24]:
# df["sample-word"] = df.sample_column.parallel_apply(function_to_apply)